In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
import keras_tuner as kt

Using TensorFlow backend


In [2]:
#  Import and read the customer_churn.csv.
attrition_df = pd.read_csv('customer_churn.csv')
attrition_df.head()

,customer_ID,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_TV,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Drop the customerID Column
attrition_df = attrition_df.drop(columns=['customer_ID'], axis=1)
attrition_df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_TV,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# Check the shape of the dataset
attrition_df.shape

(7043, 20)

In [5]:
# Check the unique values of the columns
for column in attrition_df.columns:
    print(column, attrition_df[column].unique())

gender ['Female' 'Male']
senior_citizen [0 1]
partner ['Yes' 'No']
dependents ['No' 'Yes']
tenure [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
phone_service ['No' 'Yes']
multiple_lines ['No phone service' 'No' 'Yes']
internet_service ['DSL' 'Fiber optic' 'No']
online_security ['No' 'Yes' 'No internet service']
online_backup ['Yes' 'No' 'No internet service']
device_protection ['No' 'Yes' 'No internet service']
tech_support ['No' 'Yes' 'No internet service']
streaming_TV ['No' 'Yes' 'No internet service']
streaming_movies ['No' 'Yes' 'No internet service']
contract ['Month-to-month' 'One year' 'Two year']
paperless_billing ['Yes' 'No']
payment_method ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
monthly_charges [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
total_charges 

In [6]:
# Check for missing values
attrition_df.isnull().sum()

gender               0
senior_citizen       0
partner              0
dependents           0
tenure               0
phone_service        0
multiple_lines       0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_TV         0
streaming_movies     0
contract             0
paperless_billing    0
payment_method       0
monthly_charges      0
total_charges        0
churn                0
dtype: int64

In [7]:
# Check for duplicate records
attrition_df.duplicated().sum()

22

In [8]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to the 'gender' column
attrition_df['gender'] = label_encoder.fit_transform(attrition_df['gender'])

attrition_df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_TV,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,0,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,1,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,1,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,1,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,0,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
# Convert 'total_charges' to numeric
attrition_df['total_charges'] = pd.to_numeric(attrition_df['total_charges'], errors='coerce')
attrition_df.dtypes

gender                 int32
senior_citizen         int64
partner               object
dependents            object
tenure                 int64
phone_service         object
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_TV          object
streaming_movies      object
contract              object
paperless_billing     object
payment_method        object
monthly_charges      float64
total_charges        float64
churn                 object
dtype: object

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
attrition_df = pd.get_dummies(attrition_df)
attrition_df.head()

,gender,senior_citizen,tenure,monthly_charges,total_charges,partner_No,partner_Yes,dependents_No,dependents_Yes,phone_service_No,...,contract_One year,contract_Two year,paperless_billing_No,paperless_billing_Yes,payment_method_Bank transfer (automatic),payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,churn_No,churn_Yes
0,0,0,1,29.85,29.85,0,1,1,0,1,...,0,0,0,1,0,0,1,0,1,0
1,1,0,34,56.95,1889.50,1,0,1,0,0,...,1,0,1,0,0,0,0,1,1,0
2,1,0,2,53.85,108.15,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
3,1,0,45,42.30,1840.75,1,0,1,0,1,...,1,0,1,0,1,0,0,0,1,0
4,0,0,2,70.70,151.65,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1


In [11]:
# Split our preprocessed data into our features and target arrays
X = attrition_df.drop('churn_Yes', axis=1).values
y = attrition_df['churn_Yes'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8)                 368       
                                                                 
 dense_7 (Dense)             (None, 5)                 45        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 419 (1.64 KB)
Trainable params: 419 (1.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the Sequential model together and customize metrics
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
177/177 [==============================] - 4s 6ms/step - loss: nan - accuracy: 0.7336
Epoch 2/100
177/177 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.7345
Epoch 3/100
177/177 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.7345
Epoch 4/100
177/177 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.7345
Epoch 5/100
177/177 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.7345
Epoch 6/100
177/177 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.7345
Epoch 7/100
177/177 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.7345
Epoch 8/100
177/177 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.7345
Epoch 9/100
177/177 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.7345
Epoch 10/100
177/177 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.7345
Epoch 11/100
177/17

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

45/45 - 0s - loss: nan - accuracy: 0.7353 - 367ms/epoch - 8ms/step
Loss: nan, Accuracy: 0.7352732419967651


In [21]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Save X_train_scaled.shape[1] in num_input_features since it's the size of the input features
    num_input_features = X_train_scaled.shape[1]

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=num_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [22]:
# Instantiate the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    factor=3,
    directory='my_dir',
    project_name='customer_churn')

Reloading Tuner from my_dir\customer_churn\tuner0.json


In [15]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test),batch_size=250)

Trial 26 Complete [00h 00m 14s]
val_accuracy: 0.7352732419967651

Best val_accuracy So Far: 0.7352732419967651
Total elapsed time: 00h 12m 07s
